In [ ]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Polygon
import numpy as np
from collections import namedtuple
Step=namedtuple('Step',['state','action'])

# nanshan graph

In [ ]:
shapefile_path = "./data/nanshan_grid.shp"
gdf = gpd.read_file(shapefile_path)

G = nx.Graph()

In [ ]:
for index, row in gdf.iterrows():
    current_node = tuple(row['geometry'].exterior.coords)
    fnid = row['fnid'] 
    G.add_node(current_node, fnid=fnid)

    for neighbor_index, neighbor_row in gdf.iterrows():
        neighbor_node = tuple(neighbor_row['geometry'].exterior.coords)
        if current_node != neighbor_node and Polygon(current_node).touches(Polygon(neighbor_node)):
            G.add_edge(current_node, neighbor_node)

In [ ]:
graphml_file = "./data/nanshan_network.graphml"
nx.write_graphml(G, graphml_file)

In [ ]:
G=nx.read_graphml(graphml_file)

# origin & destination coord

In [ ]:
states_tuple_path = './data/routes_states/0_0_states_tuple.npy'
state_action = np.load(states_tuple_path, allow_pickle=True)
selected_traj = np.random.choice(state_action, size=1, replace=False)[0]
origin, destination = int(selected_traj[0].state), int(selected_traj[-1].state)

# n shortest path

In [ ]:
origin_node = [(node,data) for node, data in G.nodes(data=True) if data.get('fnid') == origin]
destination_node = [(node, data) for node, data in G.nodes(data=True) if data.get('fnid') == destination]

In [ ]:
num_paths_to_find = 10
origin = origin_node[0][0]
destination = destination_node[0][0]

shortest_paths = []
for _ in range(num_paths_to_find):
    shortest_path = nx.shortest_path(G, source=origin, target=destination)
    shortest_paths.append(shortest_path)

    for u, v in zip(shortest_path[:-1], shortest_path[1:]):
        G.remove_edge(u, v)

In [ ]:
for i, path in enumerate(shortest_paths):
    fnid_values = [G.nodes[node]['fnid'] for node in path]
    print(f"shortest path {i+1}: {fnid_values}")